In [11]:
from discord_notify import send_discord_notification
from gemini import get_api_key

In [12]:
send_discord_notification("Start Quantum Circuit Artificail Intelligence")

In [13]:
GEMINI_API_KEY = get_api_key()

In [14]:
import time

In [15]:
def time_logger(original_function):
    def wrapper(*args, **kwargs):
        process_start_time = time.process_time()
        performance_start_time = time.perf_counter()

        result = original_function(*args, **kwargs)
        
        process_end_time = time.process_time()
        performance_end_time = time.perf_counter()
        execution_time = process_end_time - process_start_time
        performance_time = performance_end_time - performance_start_time
        print(f"Execution time: {execution_time:.6f} seconds", end=", ")
        print(f"Performance time: {performance_time:.6f} seconds")
        send_discord_notification(f"Execution time: {execution_time:.6f} seconds, Performance time: {performance_time:.6f} seconds")
        return result
    return wrapper

In [16]:
def Update_Model_List(_model_name, _prompt, _model, _chain_list):
    print(f"Updating model list with model: {_model_name}")
    global models_list
    if _model_name not in models_list:
        models_list.append(_model_name)
    _temp = dict()
    _temp[_model_name] = _prompt | _model
    _chain_list.append(_temp)

In [17]:
"""
def Get_Answer_caller(_chain_list, query):
    _answer_list = list()
    for models in _chain_list:
        for name, model in models.items():
            print(name, end=": ")
            process_start_time = time.process_time()
            performance_start_time = time.perf_counter()

            _answer_list.append({name : model.invoke({"question": query})})

            process_end_time = time.process_time()
            performance_end_time = time.perf_counter()
            execution_time = process_end_time - process_start_time
            performance_time = performance_end_time - performance_start_time
            print(f"Execution time: {execution_time:.4f} seconds", end=", ")
            print(f"Performance time: {performance_time:.4f} seconds")
    return _answer_list
"""

def Get_Answer_caller(query):
    _query = query
    @time_logger
    def Get_Answer(model):
        return model.invoke({"question": _query})
    return Get_Answer

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import ollama
from langchain_google_genai import GoogleGenerativeAI

chain_list = list()
model_dict = dict()
models_list = list()
for _ in ollama.list().models:
    models_list.append(_["model"])

template = """Question: {question}

Answer: Answer the question in Korean."""

prompt = ChatPromptTemplate.from_template(template)

for model_name in models_list:
    model = OllamaLLM(model=model_name)
    Update_Model_List(model_name, prompt, model, chain_list)

llm = GoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", google_api_key=GEMINI_API_KEY)
Update_Model_List("Gemini-2.5-pro", prompt, llm, chain_list)

print("Models List:")
print(", ".join(models_list))

Updating model list with model: deepseek-r1:70b
Updating model list with model: gemma3:27b
Updating model list with model: llama3.3:70b
Updating model list with model: llama3.1:70b
Updating model list with model: Gemini-2.5-pro
Models List:
deepseek-r1:70b, gemma3:27b, llama3.3:70b, llama3.1:70b, Gemini-2.5-pro


In [20]:
answer_list = list()
answer_caller = Get_Answer_caller("What is the capital of South Korea?")
for models in chain_list:
    for name, model in models.items():
        print(name, end=": ")
        send_discord_notification(f"Start {name}")
        answer = answer_caller(model)
        send_discord_notification(f"End {name}")
        answer_list.append({name: answer})
        

deepseek-r1:70b: Execution time: 0.073593 seconds, Performance time: 31.692717 seconds
gemma3:27b: Execution time: 0.010257 seconds, Performance time: 6.987489 seconds
llama3.3:70b: Execution time: 0.012517 seconds, Performance time: 10.342857 seconds
llama3.1:70b: Execution time: 0.032485 seconds, Performance time: 28.982913 seconds
Gemini-2.5-pro: Execution time: 0.016200 seconds, Performance time: 2.386745 seconds
